In [ ]:
from __future__ import annotations
import plotly.express as px
from datetime import datetime, timedelta, timezone
import mintapi
import pandas as pd
import net_worth_tracker as nwt

In [ ]:
# nwt.mint.update()

In [ ]:
data = nwt.mint.load_latest_data()
account_data = data["account_data"]
transaction_data = data["transaction_data"]
budget_data = data["budget_data"]
investments = data["investments"]

In [ ]:
account_data.availableBalance.sum()

In [ ]:
investments.plot(x="ndays", y="daily_investments")  # Saving per day

In [ ]:
budget_data.groupby("category.name").sum()

In [ ]:
nwt.mint.plot_categories(transaction_data)

In [ ]:
df = (
    transaction_data.groupby(["category.parentName", "category.name"])["amount"]
    .sum()
    .reset_index()
)
df = df[df.amount < 0].copy()
df = df[(df["category.name"] != "Transfer") & (df["category.parentName"] != "Transfer")]
df["amount"] = -df["amount"]
nwt.mint.plot_budget_spending(df)
px.sunburst(df, path=["category.parentName", "category.name"], values="amount")

In [ ]:
df = (
    transaction_data.groupby(["category.parentName", "category.name"])["amount"]
    .sum()
    .reset_index()
)
df[df.amount > 0]

In [ ]:
df[df.amount < 0]

In [ ]:
transaction_data[transaction_data["category.name"] == "Income"]

In [ ]:
account_data[account_data.type == "InvestmentAccount"]["value"].sum()

In [ ]:
average_saving_daily = investments.amount_cumsum.iloc[-1] / investments.ndays.iloc[-1]
average_saving_daily = (
    account_data[
        (account_data.type == "InvestmentAccount") & (account_data.name != "Marcella")
    ]["value"].sum()
    / investments.ndays.iloc[-1]
)
print(f"Average saving per day: {average_saving_daily}")
# Per year
average_saving_yearly = average_saving_daily * 365
print(f"Average saving per year: {average_saving_yearly}")

In [ ]:
from net_worth_tracker import retirement

defaults = dict(
    average_saving_daily=10,
    monthly_spending=700,
    start_with=2000,
    yearly_pct_raise=3,
    date_of_birth=nwt.utils.date_of_birth(),
)

future = retirement.compound(**defaults, continue_n_years_after_retirement=5)
df_future = pd.DataFrame(future).set_index("date")
future[-1]


In [ ]:
# Price vacation
n_months = retirement.cost_in_early_retirement(extra=-10_000, **defaults) * 12
print(f"Price vacation: {n_months:.1f} months in retirement.")

In [ ]:
# Difference in cost of early retirement
sims = [
    {
        "amount": amount,
        "n_months": 12 * retirement.cost_in_early_retirement(extra=amount, **defaults),
    }
    for amount in range(-50_000, 50_000, 5_000)
]
pd.DataFrame(sims).plot(x="amount", y="n_months")


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))
df_future.net_worth_inflation_corrected.plot(ax=ax)
(df_future.net_worth_inflation_corrected.max() * df_future.fire.astype(int)).plot(ax=ax)
# Plot horizontal line
target = 1_200_000  # TODO: make based on spending
ax.axhline(target, color="red", linestyle="--")

In [ ]:
pat = "PURCHASE                                AUTHORIZED ON"
to_edit = transaction_data[transaction_data.description.str.startswith(pat)]
sorted(to_edit.apply(lambda x: x.description.replace(pat, "")[9:], axis=1).to_list())